In [ ]:
#@title Default title text
pages = "" #@param {type:"string"}
days =  #@param {type:"integer"}
filter = "" #@param {type:"string"}

if not os.path.isdir('/content/facebook-scraper'):
    !git clone https://github.com/DatoJanez/facebook-scraper &> /dev/null

%cd /content/facebook-scraper
# !pip install facebook-scraper &> /dev/null
!pip install requests_html dateparser &> /dev/null

if not pages or days == 0:
    print('No page id') 

pages = pages.split(' ')

import pandas as pd
from tqdm.notebook import tqdm
import os.path
from datetime import datetime

df_initial = pd.DataFrame(columns=['page_id', 'post_id', 'text', 'post_text', 'shared_text', 'time', 'image', 'video',
       'video_thumbnail', 'video_id', 'likes', 'comments', 'shares',
       'post_url', 'link', 'user_id', 'images'])

df_initial.to_csv('all_posts.csv', index=None)

for page_id in tqdm(pages):
    !python facebook_scraper --filename {page_id}.csv --pages 100000 -d {days} {page_id} &> /dev/null 
    # !facebook-scraper --filename {page_id}.csv --pages {pages} {page_id} &> /dev/null

    if not os.path.isfile('{}.csv'.format(page_id)):
        continue
        
    df = pd.read_csv('{}.csv'.format(page_id), index_col=None)
    df = df[pd.notna(df.text) & df.text.str.contains(r'{}'.format(filter))]
    
    df["page_id"] = page_id

    df_all_posts = pd.read_csv('all_posts.csv', index_col=None)
    
    pd.concat([df_all_posts, df]).to_csv('all_posts.csv', index=None)

    !rm {page_id}.csv
    
from IPython.display import HTML
import base64

def create_download_link( title = "Download the collected data", filename = "fb-data-{}.csv".format(datetime.now().strftime('%Y-%m-%d-%H-%M'))):  
    df_all_posts = pd.read_csv('all_posts.csv', index_col=None)
    csv = df_all_posts.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link()